<br>

# Introdução

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
from paths import *

<br>

## Read Data

In [ ]:
df = pd.read_csv(
    os.path.join(output_path_tab, 'tab_daee.csv')
)
df.info()
df.head()

<br>

## Dados Espaciais

In [ ]:
# Exclui Problemas
df = df[df['erro_data'].isna()]
df = df[df['erro_coordenada'].isna()]

In [ ]:
# Checa Resultados
print(set(df['utm_mc']))
df['utm_mc'].hist()

<br>

## Concatena

In [ ]:
df_22 = df[df['utm_mc'] == 51]

geo_22 = [Point(xy) for xy in zip(df_22['utm_leste_m'], df_22['utm_norte_m'])]
gdf_22 = gpd.GeoDataFrame(
    df_22,
    crs='EPSG:31982',
    geometry=geo_22,
)

gdf_22 = gdf_22.to_crs(epsg=4326)

In [ ]:
df_23 = df[df['utm_mc'] == 45]

geo_23 = [Point(xy) for xy in zip(df_23['utm_leste_m'], df_23['utm_norte_m'])]
gdf_23 = gpd.GeoDataFrame(
    df_23,
    crs='EPSG:31983',
    geometry=geo_23,
)

gdf_23 = gdf_23.to_crs(epsg=4326)

In [ ]:
gdf = gpd.GeoDataFrame(
    pd.concat([gdf_22, gdf_23],
    ignore_index=True),
    crs=gdf_22.crs
)

<br>

## Estado

In [ ]:
datageo_path = os.path.join('~', 'Codes', 'open_geodata', 'sp_datageo')
datageo_path_data = os.path.join(datageo_path, 'data', 'output')
datageo_path_data

In [ ]:
gdf_sp = gpd.read_file(
    os.path.join(datageo_path_data, 'sp_datageo_limite_estadual.gpkg')
)
gdf_sp = gdf_sp.to_crs(epsg=3395)
gdf_sp['geometry'] = gdf_sp['geometry'].buffer(1000)
gdf_sp = gdf_sp.to_crs(epsg=4326)
#gdf_sp.explore()

In [ ]:
gdf_outorgas = gpd.clip(gdf, gdf_sp)
gdf_outorgas.plot()

<br>

## Salva arquivo

In [ ]:
gdf_outorgas.to_file(
    os.path.join(output_path_geo, 'outorgas.gpkg'),
    layer='outorgas',
    driver='GPKG'
)